#Book5_3

# 실습 18. N-GRAM 언어 모델 기본형

In [ ]:
def ngram(text, n):
  return zip(*[text[i:] for i in range(n)])

In [ ]:
print([i for i in ngram("오늘 날씨는 비", 3)])

[('오', '늘', ' '), ('늘', ' ', '날'), (' ', '날', '씨'), ('날', '씨', '는'), ('씨', '는', ' '), ('는', ' ', '비')]


In [ ]:
sentence_list = "It is raining today".split()
print(sentence_list)
print([i for i in ngram(sentence_list, 3)])

['It', 'is', 'raining', 'today']
[('It', 'is', 'raining'), ('is', 'raining', 'today')]


#한국어 처리로 코드 변경

In [ ]:
### 일본어 원서 일본어 처리 코드을 한국어 처리로 변경

!pip install konlpy # konlpy를 설치
!pip install nltk   # nltk가 설치

from konlpy.tag import Okt
from nltk import ngrams

# Okt 객체 생성
okt = Okt()

# 입력 텍스트 토큰화
sentence_list = [w for w in Okt().morphs("오늘 날씨는 비")]

# 사용 함수명이 ngram이 아니라 ngrams임에 주의
print(sentence_list)
print(["".join(li) for li in ngrams(sentence_list, 3)])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 44.9 MB/s eta 0:00:00
['오늘', '날씨', '는', '비']
['오늘날씨는', '날씨는비']


In [ ]:
from collections import defaultdict

# ngram이 아닌 ngrams를 사용함에 주의
def train(text, n):
  model = defaultdict(lambda: defaultdict(int))
  for w in ngrams(text, n):
    model[w[:-1]][w[-1]] += 1
  return model

train(sentence_list, 3)

defaultdict(<function __main__.train.<locals>.<lambda>()>,
            {('오늘', '날씨'): defaultdict(int, {'는': 1}),
             ('날씨', '는'): defaultdict(int, {'비': 1})})

In [ ]:
def predict(model, n, prefix):
  next_words = []
  for w in ngrams(prefix, n-1):
    for next_word, count in model[w].items():
      next_words.append((next_word, count))
  return sorted(next_words, key=lambda x: x[1], reverse=True)

model = train(sentence_list, 3)
predict(model, 3, sentence_list[:-1])

[('는', 1), ('비', 1)]

In [ ]:
# konlpy와 nltk가 설치되어 있지 않은 경우에는 재설치
from collections import defaultdict

from konlpy.tag import Okt
from nltk import ngrams

# 참고 이하 사용하는 ngram은 여기서 함수를 정의한 함수명임
class NgramLM:
  def __init__(self):
    self.model = defaultdict(lambda: defaultdict(int))

  def tokenize(self, text, word=True):
    if word:
      self._tokenizer = Okt().morphs
      return [w for w in self._tokenizer(text)]
    else:
      return text

  def ngram(self, text, n):
    return zip(*[text[i:] for i in range(n)])

# 이하 사용하는 tokenize는 위에 정의된 tokenize 함수(메소드)를 지칭함
  def train(self, text, n, word=True):
    if word:
      self.trained = "word"
      text = self.tokenize(text, word)
    else:
      self.trained = "char"

    for w in self.ngram(text, n):
      self.model[w[:-1]][w[-1]] += 1

  def predict(self, prefix, n, word=True):
    if self.trained == "word" and word:
      prefix = self.tokenize(prefix, word)
    elif self.trained == "char" and word:
      return "err:tokenizer of train/predict is different"

    next_words = []
    for w in self.ngram(prefix, n-1):
      for next_word, count in self.model[w].items():
        next_words.append((next_word, count))
    return sorted(next_words, key=lambda x: x[1], reverse=True)[0][0]

# 인스턴스 생성
ngram_lm = NgramLM()

# 학습(집계하여 메모리에 저장)
ngram_lm.train("나는 고양이. 나는 강아지. 나는 인간. 나는 고양이.", 3, True)

# 추론
ngram_lm.predict("나는", 3, True)

'고양이'

# 보충 수업: 파이썬 클래스 정의와 활용

In [ ]:
class Clock:
  def __init__(self):
    self.hour = 0 # 시간
    self.minute = 0 # 분

  def set_time(self, hour, minute):
    if (0 <= hour < 12) and (0 <= minute < 60):
      self.hour = hour
      self.minute = minute
      print(f"시간을{self.hour}:{self.minute:02}(으)로 설정했습니다.")
    else: print("시간 설정이 잘못되었습니다.")

  def show_time(self):
    print(f"지금 시간은 {self.hour}:{self.minute:02}입니다.")

my_clock = Clock()
my_clock.set_time(10, 30)
my_clock.show_time()

시간을10:30(으)로 설정했습니다.
지금 시간은 10:30입니다.


# 실습 19. N-GRAM 언어 모델 응용편 (고품질 데이터)

In [ ]:
!pip install datasets

from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00


In [ ]:
# 아래 코드를 실행 전 https://huggingface.co/settings/tokens 에서
# 토큰을 작성하고 나서 아래 코드 결과 화면에 뉴 토큰 복사값을 입력
from huggingface_hub import login
login()

In [ ]:
dataset = load_dataset("nlpai-lab/databricks-dolly-15k-ko")

print(dataset['train'][:1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

{'instruction': ['버진 오스트레일리아는 언제부터 운영을 시작했나요?'], 'context': ['버진 오스트레일리아는 버진 오스트레일리아 항공(Virgin Australia Airlines Pty Ltd)의 상호로, 호주에 본사를 둔 항공사입니다. 버진 브랜드를 사용하는 항공사 중 항공기 규모 면에서 가장 큰 항공사입니다. 2000년 8월 31일에 단일 노선에 두 대의 항공기로 버진 블루로 서비스를 시작했습니다.[3] 2001년 9월 호주 앤셋 항공이 파산한 후 호주 국내 시장에서 주요 항공사로 급부상했습니다. 이후 브리즈번, 멜버른, 시드니의 허브를 거점으로 호주 내 32개 도시에 직접 취항하는 항공사로 성장했습니다.[4]'], 'response': ['버진 오스트레일리아는 2000년 8월 31일에 버진 블루로 서비스를 시작했으며, 단일 노선에 두 대의 항공기를 운항했습니다.'], 'category': ['closed_qa'], 'id': [0]}


In [ ]:
strs = ""

for i, r in enumerate(dataset['train']):
  if i == 100:
    break
  strs+=r['response']

len(strs)

23840

In [ ]:
# 인스턴스화
ngram_lm = NgramLM()

# 학습(집계하여 메모리에 저장)
ngram_lm.train(strs, 3)

In [ ]:
# 추론
print(ngram_lm.predict("고양이의 애완동물", 3))
print(ngram_lm.predict("점착성이", 3))


먹이
있는


In [ ]:
max_output_tokens= 8
input_txt = "고양이의"
output_tokens = input_txt + ngram_lm.predict(input_txt, 3)

for _ in range(max_output_tokens):
  print(output_tokens)
  output_tokens += ngram_lm.predict(output_tokens, 3)

print(output_tokens)

고양이의먹이
고양이의먹이먹이
고양이의먹이먹이먹이
고양이의먹이먹이먹이먹이
고양이의먹이먹이먹이먹이먹이
고양이의먹이먹이먹이먹이먹이먹이
고양이의먹이먹이먹이먹이먹이먹이먹이
고양이의먹이먹이먹이먹이먹이먹이먹이먹이
고양이의먹이먹이먹이먹이먹이먹이먹이먹이먹이
